# Reads a GCN dataframe (4D,3D or 2D) map this dataframe with the phase information from Tariques Excel-Sheet

In [58]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *
Console_and_file_logger('merge_data/gcn_05_2020', logging.INFO)

import SimpleITK as sitk
import random
from collections import Counter
from ipywidgets import interact

%matplotlib inline
plt.rcParams.update({'font.size': 30})
%reload_ext autoreload
%autoreload 2
pd.options.display.max_columns = None

from src.utils.utils_io import Console_and_file_logger, ensure_dir
from src.visualization.Visualize import plot_3d_vol, plot_4d_vol, plot_value_histogram, show_2D_or_3D
from src.data.Dataset import get_metadata_maybe, filter_4d_vol, copy_meta_and_save, create_3d_volumes_from_4d_files, describe_sitk, describe_volume, describe_path, get_phase, is_patient_in_df, get_extremas


2020-06-12 08:33:53,592 INFO -------------------- Start --------------------
2020-06-12 08:33:53,592 INFO Working directory: /mnt/data/git/cardio.
2020-06-12 08:33:53,593 INFO Log file: ./logs/merge_data/gcn_05_2020.log
2020-06-12 08:33:53,593 INFO Log level for console: INFO


search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


# 1. Map phase and 4D dataframe

In [59]:
# read phase info
df_phase = pd.read_excel('/mnt/data/datasets/cardio/GCN/GCN Dataset_timesteps_04272020.xlsx')
print(df_phase.shape)
df_phase.head()

(725, 36)


,Study #,ID,Note,YYYY,MM,DD,Loaded to circle?,Short Axis Stack Present,SA Stack Usable? Y or why not,Axial stack present?,Contoured?,Contoured By?,I want this reviewed,Reviewed by,Exported,Zipped,Feedback,ED#,MS#,ES#,PF#,MD#,NoPhases,XML exported,Unnamed: 24,LAX_ED,LAX_ES,LAX_phases,Unnamed: 28,Unnamed: 29,Which LA used? 4Ch or Ax,Did I make the lines for insertion points?,Workspace DICOM made?,Unnamed: 33,Needs DICOMs exported,Unnamed: 35
0,1,02Z1L14N,NaN,2006.0,11.0,9.0,Y,Y,Y,NOT A CONTINOUS AX STACK,N,AP,NaN,TH,NaN,NaN,NaN,14.0,3.0,6.0,8.0,12.0,14.0,End diastole is the largest ventricular volume,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,02Z1L14N,NaN,2006.0,11.0,9.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,End systole is the smallest ventricular volume,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,04NEJQUZ,NaN,2007.0,3.0,13.0,Y,Y,Y,Y,Y,AP,NaN,TH,y,y,RV basal slice,24.0,6.0,13.0,21.0,17.0,25.0,"Peak flow is in diastole, the steepest part of...",NaN,25.0,13.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,04NEJQUZ,NaN,2008.0,3.0,14.0,Y,N,NO SA STACK,Y,N,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mid systole is halfway from ED to ES; count th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,06Z953HH,NaN,2007.0,12.0,3.0,Y,N,NO SA STACK,Y,N,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mid diastole is the phase before the atrium st...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# read 4D dataframe
df_4d = pd.read_csv('data/raw/gcn_05_2020_ax_sax_86/SAX_4D_dataset.csv')
print(df_4d.shape)
df_4d.head()

(86, 5)


,patient,x_path,y_path,timesteps,img_shape
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 10, 15, 21, 24]","(5, 10, 256, 256)"
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[0, 3, 7, 11, 13]","(5, 12, 256, 256)"
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[3, 6, 11, 19, 24]","(5, 13, 288, 288)"
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[4, 8, 14, 22, 23]","(5, 14, 288, 288)"
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 12, 18, 22, 24]","(5, 16, 320, 320)"


In [61]:
df_4d['patient'] = df_4d['patient'].str.upper()

In [62]:
# extract the date from the filenames
temp = df_4d.x_path[0]
def extract_date(row):
    """
    helper to extract the date as tuple from the filename
    """
    date = os.path.basename(row['x_path']).split('_')[1].split('-')
    return int(date[0]), int(date[1]), int(date[2])

df_4d['YYYY'],df_4d['MM'], df_4d['DD'] = zip(*df_4d.apply(extract_date, axis=1))
df_4d.head()

,patient,x_path,y_path,timesteps,img_shape,YYYY,MM,DD
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 10, 15, 21, 24]","(5, 10, 256, 256)",2007,5,23
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[0, 3, 7, 11, 13]","(5, 12, 256, 256)",2005,6,27
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[3, 6, 11, 19, 24]","(5, 13, 288, 288)",2007,2,13
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[4, 8, 14, 22, 23]","(5, 14, 288, 288)",2007,12,10
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 12, 18, 22, 24]","(5, 16, 320, 320)",2007,5,22


In [63]:
# left outer join of the metadata into the 4d dataframe, based on the patient id
# Check if this file has no dublicated patient ids
len(df_4d.patient.unique()) == len(df_4d)

True

## Drop rows in the phase excel sheet without a phase mapping
## Make sure the Patient ID is with upper letters

In [64]:
# minor cleaning of the df_meta, some patients have multiple studies with the same patient id and date
# Merge will result in more rows, because of multi mapping
# How many rows with a short axis stack do we have?
#df_phase_cleaned = df_phase[df_phase['Short Axis Stack Present'].notnull()]
#df_phase_cleaned = df_phase_cleaned[df_phase_cleaned['Contoured?']=='Y']
df_phase_cleaned = df_phase[df_phase['ED#'].notnull()].copy()
#df_phase_cleaned = df_phase_cleaned[df_phase_cleaned['Exported'].notnull()]
#df_phase_cleaned = df_phase
df_phase_cleaned['ID'] = df_phase_cleaned['ID'].str.upper()
print(df_phase_cleaned.shape)
df_phase_cleaned.head()


(344, 36)


,Study #,ID,Note,YYYY,MM,DD,Loaded to circle?,Short Axis Stack Present,SA Stack Usable? Y or why not,Axial stack present?,Contoured?,Contoured By?,I want this reviewed,Reviewed by,Exported,Zipped,Feedback,ED#,MS#,ES#,PF#,MD#,NoPhases,XML exported,Unnamed: 24,LAX_ED,LAX_ES,LAX_phases,Unnamed: 28,Unnamed: 29,Which LA used? 4Ch or Ax,Did I make the lines for insertion points?,Workspace DICOM made?,Unnamed: 33,Needs DICOMs exported,Unnamed: 35
0,1,02Z1L14N,NaN,2006.0,11.0,9.0,Y,Y,Y,NOT A CONTINOUS AX STACK,N,AP,NaN,TH,NaN,NaN,NaN,14.0,3.0,6.0,8.0,12.0,14.0,End diastole is the largest ventricular volume,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,04NEJQUZ,NaN,2007.0,3.0,13.0,Y,Y,Y,Y,Y,AP,NaN,TH,y,y,RV basal slice,24.0,6.0,13.0,21.0,17.0,25.0,"Peak flow is in diastole, the steepest part of...",NaN,25.0,13.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,0AE4R74L,NaN,1900.0,1.0,1.0,Y,Y,Y,N,Y,AP,NaN,TH,y,y,NaN,13.0,2.0,4.0,7.0,11.0,13.0,Y,NaN,13.0,5.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,0HQQW4ZN,- Lax missing,2007.0,5.0,23.0,Y,Y,Y,Y,Y,AP,NaN,TH,y,y,NaN,25.0,7.0,11.0,16.0,22.0,25.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,0PTV75MP,NaN,2005.0,6.0,27.0,Y,Y,Y,N,Y,AP,NaN,TH,y,y,NaN,1.0,4.0,8.0,12.0,14.0,14.0,Y,NaN,1.0,9.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# compare the patient ids
patient_dicom, patient_phase = set(df_4d.patient.values), set(df_phase_cleaned.ID.values)
print('dicom patients: {}'.format(len(patient_dicom)))
print('phase patients: {}'.format(len(patient_phase)))
print('dicom patients, that are not in the excel-sheet: {}'.format(len(patient_dicom - patient_phase)))

dicom patients: 86
phase patients: 332
dicom patients, that are not in the excel-sheet: 0


In [66]:
# show the patients that are not in the phase excel sheet
patient_dicom - patient_phase

set()

## Inner join of the phase excel-sheet and the 4D dataframe

In [67]:
# merge the 4D dataframe with the cleaned phase info
# drop all rows without a short axis stack/ which are not contoured
df_merge = df_4d.merge(df_phase_cleaned, how='inner', left_on=['patient','YYYY'], right_on=['ID','YYYY'])
print(df_merge.shape)
df_merge.head()

(86, 43)


,patient,x_path,y_path,timesteps,img_shape,YYYY,MM_x,DD_x,Study #,ID,Note,MM_y,DD_y,Loaded to circle?,Short Axis Stack Present,SA Stack Usable? Y or why not,Axial stack present?,Contoured?,Contoured By?,I want this reviewed,Reviewed by,Exported,Zipped,Feedback,ED#,MS#,ES#,PF#,MD#,NoPhases,XML exported,Unnamed: 24,LAX_ED,LAX_ES,LAX_phases,Unnamed: 28,Unnamed: 29,Which LA used? 4Ch or Ax,Did I make the lines for insertion points?,Workspace DICOM made?,Unnamed: 33,Needs DICOMs exported,Unnamed: 35
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 10, 15, 21, 24]","(5, 10, 256, 256)",2007,5,23,7,0HQQW4ZN,- Lax missing,5.0,23.0,Y,Y,Y,Y,Y,AP,NaN,TH,y,y,NaN,25.0,7.0,11.0,16.0,22.0,25.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[0, 3, 7, 11, 13]","(5, 12, 256, 256)",2005,6,27,8,0PTV75MP,NaN,6.0,27.0,Y,Y,Y,N,Y,AP,NaN,TH,y,y,NaN,1.0,4.0,8.0,12.0,14.0,14.0,Y,NaN,1.0,9.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[3, 6, 11, 19, 24]","(5, 13, 288, 288)",2007,2,13,10,0RPELLU8,- LAX missing,2.0,13.0,Y,Y,Y,Y,Y,SP,NaN,TH,y,y,NaN,25.0,4.0,7.0,12.0,20.0,25.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[4, 8, 14, 22, 23]","(5, 14, 288, 288)",2007,12,10,12,11YU3CUF,NaN,12.0,10.0,Y,Y,Y,Y,Y,SP,NaN,TH,y,y,NaN,24.0,5.0,9.0,15.0,23.0,25.0,NaN,NaN,1.0,12.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 12, 18, 22, 24]","(5, 16, 320, 320)",2007,5,22,13,13JLP3HN,NaN,5.0,22.0,Y,Y,Y,Y,Y,AP,NaN,TH,y,y,NaN,25.0,7.0,13.0,19.0,23.0,25.0,Y,NaN,1.0,13.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# check if we have a row with ED# == Nan
# this number reflects a dicom stack where we didn't find the phase
# patient 02Z1L14N, dicom exists, excel column exported == NAN
# patient RQ1JAJAU, dicom eists, excel column exported == NAN
# patient 778N33YG, dicom from 2005, phase from 2006
# patient RQ1JAJAU, dicom from 2007, phase from 2006
print(df_merge['ED#'].isnull().sum())
df_merge[df_merge['ED#'].isnull()]

0


,patient,x_path,y_path,timesteps,img_shape,YYYY,MM_x,DD_x,Study #,ID,Note,MM_y,DD_y,Loaded to circle?,Short Axis Stack Present,SA Stack Usable? Y or why not,Axial stack present?,Contoured?,Contoured By?,I want this reviewed,Reviewed by,Exported,Zipped,Feedback,ED#,MS#,ES#,PF#,MD#,NoPhases,XML exported,Unnamed: 24,LAX_ED,LAX_ES,LAX_phases,Unnamed: 28,Unnamed: 29,Which LA used? 4Ch or Ax,Did I make the lines for insertion points?,Workspace DICOM made?,Unnamed: 33,Needs DICOMs exported,Unnamed: 35


In [69]:
# check for double patient ids in the merged dataframe
# This means we mapped to rows from the timesteps xls
print(len(df_merge.patient.unique()))
from collections import Counter
c = Counter(df_merge.patient.values)
print(sorted(df_merge.patient.values, key=c.get, reverse=True)[:5])

86
['0HQQW4ZN', '0PTV75MP', '0RPELLU8', '11YU3CUF', '13JLP3HN']


In [70]:
df_merge.head()

,patient,x_path,y_path,timesteps,img_shape,YYYY,MM_x,DD_x,Study #,ID,Note,MM_y,DD_y,Loaded to circle?,Short Axis Stack Present,SA Stack Usable? Y or why not,Axial stack present?,Contoured?,Contoured By?,I want this reviewed,Reviewed by,Exported,Zipped,Feedback,ED#,MS#,ES#,PF#,MD#,NoPhases,XML exported,Unnamed: 24,LAX_ED,LAX_ES,LAX_phases,Unnamed: 28,Unnamed: 29,Which LA used? 4Ch or Ax,Did I make the lines for insertion points?,Workspace DICOM made?,Unnamed: 33,Needs DICOMs exported,Unnamed: 35
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 10, 15, 21, 24]","(5, 10, 256, 256)",2007,5,23,7,0HQQW4ZN,- Lax missing,5.0,23.0,Y,Y,Y,Y,Y,AP,NaN,TH,y,y,NaN,25.0,7.0,11.0,16.0,22.0,25.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[0, 3, 7, 11, 13]","(5, 12, 256, 256)",2005,6,27,8,0PTV75MP,NaN,6.0,27.0,Y,Y,Y,N,Y,AP,NaN,TH,y,y,NaN,1.0,4.0,8.0,12.0,14.0,14.0,Y,NaN,1.0,9.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[3, 6, 11, 19, 24]","(5, 13, 288, 288)",2007,2,13,10,0RPELLU8,- LAX missing,2.0,13.0,Y,Y,Y,Y,Y,SP,NaN,TH,y,y,NaN,25.0,4.0,7.0,12.0,20.0,25.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[4, 8, 14, 22, 23]","(5, 14, 288, 288)",2007,12,10,12,11YU3CUF,NaN,12.0,10.0,Y,Y,Y,Y,Y,SP,NaN,TH,y,y,NaN,24.0,5.0,9.0,15.0,23.0,25.0,NaN,NaN,1.0,12.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 12, 18, 22, 24]","(5, 16, 320, 320)",2007,5,22,13,13JLP3HN,NaN,5.0,22.0,Y,Y,Y,Y,Y,AP,NaN,TH,y,y,NaN,25.0,7.0,13.0,19.0,23.0,25.0,Y,NaN,1.0,13.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# find patients with labelled timesteps != 5

In [72]:
def get_number_of_contours(t_list):
    import ast
    return len(ast.literal_eval(t_list))
n_contours = df_merge.timesteps.apply(get_number_of_contours)


In [73]:
c = Counter(n_contours)

In [74]:
c

Counter({5: 85, 4: 1})

In [75]:
# save merged 4D dataframe
df_4d.to_csv('data/raw/gcn_05_2020_ax_sax_86/SAX_4D_merged_with_phase.csv')

# 2. Map phase and 3D dataframe

In [76]:
# load both dataframes
# read phase info
df_phase = pd.read_excel('/mnt/data/datasets/cardio/GCN/GCN Dataset_timesteps_04272020.xlsx')
print(df_phase.shape)
df_phase.head()

(725, 36)


,Study #,ID,Note,YYYY,MM,DD,Loaded to circle?,Short Axis Stack Present,SA Stack Usable? Y or why not,Axial stack present?,Contoured?,Contoured By?,I want this reviewed,Reviewed by,Exported,Zipped,Feedback,ED#,MS#,ES#,PF#,MD#,NoPhases,XML exported,Unnamed: 24,LAX_ED,LAX_ES,LAX_phases,Unnamed: 28,Unnamed: 29,Which LA used? 4Ch or Ax,Did I make the lines for insertion points?,Workspace DICOM made?,Unnamed: 33,Needs DICOMs exported,Unnamed: 35
0,1,02Z1L14N,NaN,2006.0,11.0,9.0,Y,Y,Y,NOT A CONTINOUS AX STACK,N,AP,NaN,TH,NaN,NaN,NaN,14.0,3.0,6.0,8.0,12.0,14.0,End diastole is the largest ventricular volume,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,02Z1L14N,NaN,2006.0,11.0,9.0,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,End systole is the smallest ventricular volume,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,04NEJQUZ,NaN,2007.0,3.0,13.0,Y,Y,Y,Y,Y,AP,NaN,TH,y,y,RV basal slice,24.0,6.0,13.0,21.0,17.0,25.0,"Peak flow is in diastole, the steepest part of...",NaN,25.0,13.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,04NEJQUZ,NaN,2008.0,3.0,14.0,Y,N,NO SA STACK,Y,N,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mid systole is halfway from ED to ES; count th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,06Z953HH,NaN,2007.0,12.0,3.0,Y,N,NO SA STACK,Y,N,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mid diastole is the phase before the atrium st...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
df_3d = pd.read_csv('data/raw/gcn_05_2020_ax_sax_86/SAX_3D_dataset.csv')
print(df_3d.shape)
df_3d.head()

(429, 3)


,patient,x_path,y_path
0,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
1,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
2,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
3,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
4,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...


## Extend the 3D dataframe by timestep and the date

In [78]:
# extract t from the filenames
def extract_t_from_filename(f_name):
    return int(os.path.basename(os.path.normpath(f_name)).split('__')[1].split('_')[0].replace('t',''))
df_3d['t'] = df_3d.x_path.apply(extract_t_from_filename)
df_3d.head()

,patient,x_path,y_path,t
0,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10
1,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,15
2,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,21
3,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24
4,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,6


In [79]:
# extract the date from the filenames
def extract_date(row):
    """
    helper to extract the date as tuple from the filename
    """
    date = os.path.basename(row['x_path']).split('_')[1].split('-')
    return int(date[0]), int(date[1]), int(date[2])

df_3d['YYYY'],df_3d['MM'], df_3d['DD'] = zip(*df_3d.apply(extract_date, axis=1))
df_3d['patient'] = df_3d['patient'].apply(lambda x : x.split('-')[1])
print(df_3d.shape)
df_3d.head(20)

(429, 7)


,patient,x_path,y_path,t,YYYY,MM,DD
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10,2007,5,23
1,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,15,2007,5,23
2,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,21,2007,5,23
3,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,23
4,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,6,2007,5,23
5,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,0,2005,6,27
6,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,11,2005,6,27
7,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,13,2005,6,27
8,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,3,2005,6,27
9,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,7,2005,6,27


## Minor cleaning of both dataframes

In [80]:
# minor cleaning and dropping of the phase dataframe
# drop rows without Phase info
# clean empty spaces in ID column, transform to upper
df_phase_cleaned = df_phase[df_phase['ED#'].notnull()].copy()
df_phase_cleaned['ID'] = df_phase_cleaned['ID'].apply(lambda x: str(x).replace(' ', ''))
df_phase_cleaned['ID'] = df_phase_cleaned['ID'].str.upper()
print('Cleaned phase dataframe: {}'.format(df_phase_cleaned.shape))

df_3d['patient'] = df_3d['patient'].apply(lambda x: str(x).replace(' ', ''))
df_3d['patient'] = df_3d['patient'].str.upper()
print('Cleaned dicom dataframe: {}'.format(df_3d.shape))

Cleaned phase dataframe: (344, 36)
Cleaned dicom dataframe: (429, 7)


## Inner join of the 3D dataframe and the phase info

In [81]:
df_merge = df_3d.merge(df_phase_cleaned[['ID','YYYY', 'ED#', 'MS#', 'ES#', 'PF#', 'MD#']], how='inner', left_on=['patient','YYYY'], right_on=['ID','YYYY'])
print(df_merge.shape)
df_merge.head()

(429, 13)


,patient,x_path,y_path,t,YYYY,MM,DD,ID,ED#,MS#,ES#,PF#,MD#
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0
1,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,15,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0
2,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,21,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0
3,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0
4,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,6,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0


## For each row find the matching phase

In [82]:
def get_phase(row):
    # different index start (0 or 1), need to increase the timestep index by 1
    phase = 'PHASE_NOT_MAPPED'
    try:
        t_norm = int(row.t + 1)
        if int(row['ED#']) == t_norm:
            phase = 'ED'
        if int(row['MS#']) == t_norm:
            phase = 'MS'
        if int(row['ES#']) == t_norm:
            phase = 'ES'
        if int(row['PF#']) == t_norm:
            phase = 'PF'
        if int(row['MD#']) == t_norm:
            phase = 'MD'
        return phase
    except Exception as e:
        print(t_norm)
        print(str(e))
        return phase
df_merge['phase'] = df_merge.apply(get_phase, axis=1)
df_merge.head(20)


,patient,x_path,y_path,t,YYYY,MM,DD,ID,ED#,MS#,ES#,PF#,MD#,phase
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,ES
1,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,15,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,PF
2,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,21,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,MD
3,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,ED
4,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,6,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,MS
5,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,0,2005,6,27,0PTV75MP,1.0,4.0,8.0,12.0,14.0,ED
6,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,11,2005,6,27,0PTV75MP,1.0,4.0,8.0,12.0,14.0,PF
7,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,13,2005,6,27,0PTV75MP,1.0,4.0,8.0,12.0,14.0,MD
8,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,3,2005,6,27,0PTV75MP,1.0,4.0,8.0,12.0,14.0,MS
9,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,7,2005,6,27,0PTV75MP,1.0,4.0,8.0,12.0,14.0,ES


## Check if we mapped a phase to each row

In [83]:
pd.value_counts(df_merge.phase)

PF    86
MS    86
ED    86
ES    86
MD    85
Name: phase, dtype: int64

In [84]:
# show rows without phase info
df_merge[df_merge['phase']=='PHASE_NOT_MAPPED']

,patient,x_path,y_path,t,YYYY,MM,DD,ID,ED#,MS#,ES#,PF#,MD#,phase


In [85]:
@interact
def filter_df(search_str=''):
    search_str = search_str.upper()
    print(search_str)
    return df_merge[df_merge['patient'].str.contains(search_str)]

interactive(children=(Text(value='', description='search_str'), Output()), _dom_classes=('widget-interact',))

In [86]:
# save the 3D dataframe with phase info to disk
df_merge.to_csv('data/raw/gcn_05_2020_ax_sax_86/SAX_3D_merged_with_phase.csv', index=False)